In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import xgboost as xgb
pd.set_option('display.max_colwidth', -1)

In [2]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [3]:
from utils import UtilsKy, Statistic

In [16]:
# kyw3
path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str)
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

db_teach = pd.read_csv(path_data + 'db_teach_kyw3_with_zip_factors.csv', dtype=str)
db_test = pd.read_csv(path_data + 'db_test_kyw3_with_zip_factors.csv', dtype=str)

In [117]:
# ky9
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# for prod
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_FOR_PROD, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9_FOR_PROD , dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# all false
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_ALL, dtype=str)

# 300th in teach (10%) 
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_ALL , dtype=str)

In [25]:
db_teach.columns

Index(['amount', 'bank_currency', 'bin', 'day_of_week', 'hour', 'id',
       'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status',
       'distance_haversine', 'distance_l2', 'distance_exp', 'is_city_resolved',
       'ip', 'network', 'num_network', 'phone_3_norm', 'gender',
       'amount_deviation', 'is_gender_undef', 'is_city_undef',
       'is_zip_numeric', 'is_len_zip_standart'],
      dtype='object')

In [6]:
db_teach.shape[0]

433425

In [7]:
db_teach[db_teach.status.isin([1, '1', 'true'])].shape[0]

6261

In [116]:
COL_FACTORS = [ 'amount', 'bank_currency', 'hour', 'day_of_week', 
               'bin', 'longitude', 'latitude', 'phone_2_norm', 'is_gender_undef', 'is_city_undef']

In [117]:
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [118]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [119]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [120]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [121]:
train = train.values
test = test.values
label = db_teach.status

In [122]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [123]:
# result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [124]:
# Kyw3
max_depths =[3]
nrounds = [80, 90]
etas = [0.2, 0.35]

In [125]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth': max_depth, 'learning_rate': eta
                      , 'n_estimators': nround
                     }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]

            description = '-' . join([str(elem) for elem in (max_depth, nround, eta, 'is_gen_und', 'is_city_undef')])                    
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [126]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [127]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
76,3-90-0.2-is_gen_und-is_city_undef,9.07,14.38,15.73,19.90,22.01,23.56,25.79,30.69,42.82,154.60
68,3-90-0.2-is_gen_und-is_city_undef-is_zip_numeric,9.02,13.96,15.97,19.84,20.74,21.39,22.65,28.13,44.71,152.37
72,3-80-0.2-is_gen_und-is_city_undef,7.34,13.32,15.53,19.15,21.86,23.51,25.75,30.83,43.47,151.50
40,3-80-0.2-is_gen_und,7.99,14.26,17.23,19.20,19.24,23.05,23.28,27.74,43.46,149.12
56,3-80-0.2-is_gen_und-is_zip_numeric,6.35,11.85,16.67,17.96,21.41,23.75,24.94,28.78,45.67,148.69
32,3-80-0.2-is_gen_und-2_is_zip,6.35,9.90,16.56,18.32,21.68,23.90,25.02,29.22,46.24,148.27
52,3-90-0.2-is_gen_und-is_len_zip_standart,6.42,12.24,16.27,19.61,19.99,21.78,23.28,29.71,43.95,148.19
44,3-90-0.2-is_gen_und,9.09,12.50,17.48,19.20,20.26,22.23,22.63,27.44,42.20,148.17
48,3-80-0.2-is_gen_und-is_len_zip_standart,6.51,12.05,14.43,18.95,20.85,22.39,25.59,30.59,44.55,147.93
60,3-90-0.2-is_gen_und-is_zip_numeric,7.14,13.49,16.32,17.63,20.16,21.90,22.40,27.12,44.86,146.72
